In [210]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, sum, lit, regexp_extract, initcap, when, mean, date_format, avg, concat,max as spark_max, current_date
from pyspark.sql import types as T
from pyspark.sql import functions as F
import logging
import warnings
warnings.filterwarnings('ignore')


# Data Preprocessing: Address any inconsistencies or errors found during validation and clean the data as necessary.

## Data Cleaning Process

This outlines the steps involved in cleaning and preparing the customer data for further analysis. The process includes handling missing values, detecting and removing duplicates, formatting phone numbers, and ensuring proper capitalization in specified columns.

## Steps Overview

### Step 1: Count Missing Values Before Filling

Count the missing (null) values in each column before applying any filling operations to understand the extent of missing data.

### Step 2: Count Duplicates Before Dropping

Check for duplicate records in each column to identify potential data integrity issues before removing duplicates.

### Step 3: Format and Clean Phone Numbers

Process phone numbers to remove any formatting issues and append the correct country code using the provided country codes file.

### Step 4: Drop Duplicates Based on Primary Key

Remove duplicate records based on the Primary Key` column to ensure each customer is uniquely represented.

### Step 5: Fill Missing Values in 'Email' and 'Phone' Columns

Fill missing values in the `Email` and `Phone` columns with a default value of `'unknown'` to ensure completeness.

### Step 6: Capitalize the First Letter in 'Name' and 'Country' Columns

Capitalize the first letter of each word in the `Name` and `Country` columns for consistent formatting.

### Step 7: Cross-Validation - Count Missing Values After Filling

Recount the missing values in each column after the filling operation to validate that missing data has been appropriately handled.

### Step 8: Count Duplicates After Dropping

Recheck for duplicates in each column after the removal process to confirm all duplicates have been successfully dropped.

### Step 9: Export the Cleaned Data to a CSV File

Finally, export the cleaned DataFrame to a CSV file for further use.

---


# Reusable Data Cleaning Functions

In [211]:
%run utility/common_utility.ipynb

In [212]:
import logging
import os

class CustomFormatter(logging.Formatter):
    # ANSI escape codes for colors
    GREEN = "\033[92m"
    RESET = "\033[0m"

    def format(self, record):
        # Add green color to INFO level messages
        if record.levelno == logging.INFO:
            record.msg = f"{self.GREEN}{record.msg}{self.RESET}"
        return super().format(record)

# Ensure the logs directory exists
os.makedirs('logs', exist_ok=True)

# Initialize the logger
logger = logging.getLogger('DataCleaningLogger')
logger.setLevel(logging.INFO)

# Remove all existing handlers
for handler in logger.handlers[:]:
    logger.removeHandler(handler)

# Create a stream handler
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(CustomFormatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))

# Create a file handler
fh = logging.FileHandler('logs/data_cleaning.log')
fh.setLevel(logging.INFO)
fh.setFormatter(CustomFormatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))

# Add handlers to the logger
logger.addHandler(ch)
logger.addHandler(fh)

In [214]:
# Step 1: Load data from CSV files into DataFrames using PySpark
logger.info("Step 1: Loading data from CSV files into DataFrames...")

# Define a list of tuples mapping file names to DataFrame variables
file_names = [
    ("customers_df", "Dataset/customers.csv"),
    ("products_df", "Dataset/products.csv"),
    ("transactions_df", "Dataset/transactions.csv"),
    ("interactions_df", "Dataset/interactions.csv"),
    ("sales_team_df", "Dataset/sales_team.csv")
]

# Load all DataFrames using a loop
dfs = {}
for df_var, file in file_names:
    dfs[df_var] = spark.read.csv(file, header=True, inferSchema=True)

# Assign DataFrames to their respective variables
customers_df = dfs["customers_df"]
products_df = dfs["products_df"]
transactions_df = dfs["transactions_df"]
interactions_df = dfs["interactions_df"]
sales_team_df = dfs["sales_team_df"]

# Step 2: Initial Validation - Display records for each DataFrame
logger.info("Step 2: Displaying initial records for validation...")

# Loop through the DataFrames to display the first 5 records
for df_var, _ in file_names:
    logger.info(f"{df_var.replace('_df', '').capitalize()} DataFrame:")
    dfs[df_var].show(5, truncate=False)


2024-09-02 16:49:53,784 - DataCleaningLogger - INFO - Step 1: Loading data from CSV files into DataFrames...
2024-09-02 16:49:54,147 - DataCleaningLogger - INFO - Step 2: Displaying initial records for validation...
2024-09-02 16:49:54,149 - DataCleaningLogger - INFO - Customers DataFrame:
2024-09-02 16:49:54,174 - DataCleaningLogger - INFO - Products DataFrame:
2024-09-02 16:49:54,209 - DataCleaningLogger - INFO - Transactions DataFrame:
2024-09-02 16:49:54,243 - DataCleaningLogger - INFO - Interactions DataFrame:
2024-09-02 16:49:54,282 - DataCleaningLogger - INFO - Sales_team DataFrame:


+------------------------------------+-----------------+----------------------+-------------+----------------+
|Customer_ID                         |Name             |Email                 |Phone        |Country         |
+------------------------------------+-----------------+----------------------+-------------+----------------+
|a85e6a90-78d5-490c-a53f-c58b2e57c59b|Shannon Deleon   |NULL                  |5878628895   |Japan           |
|babec972-ffb3-4c56-99c3-e8e3855adf0f|Christina Sanchez|craigprice@example.org|4832368495   |Haiti           |
|d74c33bd-69d9-4718-9e00-d1895a41ddac|Thomas Brown     |vjohnson@example.org  |(276)903-7065|Pakistan        |
|ff05ceba-f459-4714-a252-e03198d9934c|Lindsey Bradford |kathryn50@example.net |NULL         |Marshall Islands|
|f20755f6-8481-4904-afe6-504451ceded5|John Boyer       |jennifer15@example.org|(749)644-5721|New Caledonia   |
+------------------------------------+-----------------+----------------------+-------------+----------------+
o

# Data Cleaning

## 1. Customers Table

In [215]:
# Step 1: Count missing (null) values for each column before filling them
logger.info("Step 1: Counting missing values in each column before filling them...")
missing_values_before = count_missing_values(customers_df)
missing_values_before.show()

# Step 2: Count duplicates before dropping them
logger.info("Step 2: Checking for duplicates in each column before dropping them...")
duplicate_count = count_duplicates_per_column(customers_df)
duplicate_count.show()

# Step 3: Format and clean phone numbers
logger.info("Step 3: Formatting and cleaning phone numbers...")
cleaned_customers_df = process_phone_numbers(customers_df, "Dataset/countries.csv")
print("Completed.")

# Step 4: Drop duplicates based on 'Customer_ID' if any are found
logger.info("Step 4: Checking and removing duplicate records based on 'Customer_ID'...")
cleaned_customers_df = drop_duplicates(cleaned_customers_df, "Customer_ID")

# Step 5: Format and clean Email
logger.info("Step 5: Formatting and cleaning Email...")
cleaned_customers_df = validate_emails(cleaned_customers_df, "Email")
print("Completed.")

# Step 5: Fill missing values in 'Email' and 'Phone' columns
logger.info("Step 5: Filling missing values in 'Email' and 'Phone' columns...")
cleaned_customers_df = fill_missing_values(cleaned_customers_df, {'Email': 'unknown', 'Phone': 'unknown'})
cleaned_customers_df.show(5, truncate=False)

# Step 6: Capitalize the first letter of the first and last names in the 'Name' and 'Country' columns
logger.info("Step 6: Capitalizing the first letter of each word in the 'Name' and 'Country' columns...")
cleaned_customers_df = capitalize_columns(cleaned_customers_df, ["Name", "Country"])
print("Completed.")

# Step 7: Cross-validation - Count missing values again after filling them
logger.info("Step 7: Counting missing values in each column after filling them...")
missing_values_after = count_missing_values(cleaned_customers_df)
missing_values_after.show()

# Step 8: Count duplicates after dropping them
logger.info("Step 8: Checking for duplicates in each column after dropping them...")
duplicate_count_after = count_duplicates_per_column(cleaned_customers_df)
duplicate_count_after.show()

# Step 9: Export the cleaned data to a CSV file
logger.info("Step 9: Exporting the cleaned data to 'cleaned_customers.csv'...")
# save_df_to_csv(cleaned_customers_df, "Cleaned_data/cleaned_customers.csv")

# Display the count of records after phone number processing
record_count_after_cleaning = cleaned_customers_df.count()
print(f"Number of records after cleaning: {record_count_after_cleaning}")

logger.info("Data cleaning and export completed successfully.")

2024-09-02 16:49:54,322 - DataCleaningLogger - INFO - Step 1: Counting missing values in each column before filling them...
2024-09-02 16:49:54,434 - DataCleaningLogger - INFO - Step 2: Checking for duplicates in each column before dropping them...


+-----------+----+-----+-----+-------+
|Customer_ID|Name|Email|Phone|Country|
+-----------+----+-----+-----+-------+
|          0|   0|   52|   52|      0|
+-----------+----+-----+-----+-------+



2024-09-02 16:49:55,524 - DataCleaningLogger - INFO - Step 3: Formatting and cleaning phone numbers...
2024-09-02 16:49:55,664 - DataCleaningLogger - INFO - Step 4: Checking and removing duplicate records based on 'Customer_ID'...


+-----------+---------------+
|     Column|Duplicate_Count|
+-----------+---------------+
|Customer_ID|             25|
|       Name|             26|
|      Email|             22|
|      Phone|             23|
|    Country|            157|
+-----------+---------------+

Completed.


2024-09-02 16:49:55,976 - DataCleaningLogger - INFO - Step 5: Formatting and cleaning Email...


...

All non-null emails are valid.
Completed.
+------------------------------------+-----------------+----------------------+---------------+-----------+
|Customer_ID                         |Name             |Email                 |Phone          |Country    |
+------------------------------------+-----------------+----------------------+---------------+-----------+
|003ca69a-991c-4c11-899a-51bb7365499d|Erica Diaz       |hurleyanna@example.com|+964-8545044941|Iraq       |
|006af455-013b-4c09-a6df-15ca3d41010f|Jason Jackson    |dylanduran@example.com|+63-2484285464 |Philippines|
|00fc38f7-b5c7-465c-839b-a55185f2635f|Heather Schneider|larajohn@example.org  |+264-7935879728|Namibia    |
|015cc4e1-5cf8-441c-80ad-ca3536c53e9a|Matthew Wilson   |timothyho@example.org |+357-8883554148|Cyprus     |
|01d13428-d511-4c3d-90c7-b5624931ff48|Robert Contreras |lydia12@example.com   |unknown        |Swaziland  |
+------------------------------------+-----------------+----------------------+---------------

2024-09-02 16:49:56,603 - DataCleaningLogger - INFO - Step 8: Checking for duplicates in each column after dropping them...


+-----------+----+-----+-----+-------+
|Customer_ID|Name|Email|Phone|Country|
+-----------+----+-----+-----+-------+
|          0|   0|    0|    0|      0|
+-----------+----+-----+-----+-------+



2024-09-02 16:49:57,638 - DataCleaningLogger - INFO - Step 9: Exporting the cleaned data to 'cleaned_customers.csv'...
2024-09-02 16:49:57,746 - DataCleaningLogger - INFO - Data cleaning and export completed successfully.


+-----------+---------------+
|     Column|Duplicate_Count|
+-----------+---------------+
|Customer_ID|              0|
|       Name|              1|
|      Email|              1|
|      Phone|              1|
|    Country|            151|
+-----------+---------------+

Number of records after cleaning: 500


## 2. Products Table

In [216]:
# Step 1: Count missing (null) values for each column before filling them
logger.info("Step 1: Counting missing values in each column before filling them...")
missing_values_before = count_missing_values(products_df)
missing_values_before.show()

# Step 2: Count duplicates before dropping them
logger.info("Step 2: Checking for duplicates in each column before dropping them...")
duplicate_count = count_duplicates_per_column(products_df)
duplicate_count.show()

# Step 3: Drop duplicates based on 'Customer_ID' if any are found
logger.info("Step 3: Checking and removing duplicate records based on 'Customer_ID'...")
cleaned_products_df = drop_duplicates(products_df, "Product_ID")

# Step 4: Fill missing values in 'Email' and 'Phone' columns
logger.info("Step 4: Filling missing values in 'Email' and 'Phone' columns...")
cleaned_products_df = fill_missing_values(cleaned_products_df, {"Category": "Uncategorized"})
cleaned_products_df.show(5, truncate=False)

# Step 5: Capitalize the first letter of the first and last names in the 'Name' and 'Country' columns
logger.info("Step 5: Capitalizing the first letter of each word in the 'Product_Name' and 'Category' columns...")
cleaned_products_df = capitalize_columns(cleaned_products_df, ["Product_Name", "Category"])
print("Completed.")

# Step 6: Cross-validation - Count missing values again after filling them
logger.info("Step 6: Counting missing values in each column after filling them...")
missing_values_after = count_missing_values(cleaned_products_df)
missing_values_after.show()

# Step 7: Count duplicates after dropping them
logger.info("Step 7: Checking for duplicates in each column after dropping them...")
duplicate_count_after = count_duplicates_per_column(cleaned_products_df)
duplicate_count_after.show()

# Step 8: Handle negative or zero prices by replacing with average price
logger.info("Step 8: Replacing negative or zero prices with the average price...")
avg_price = products_df.agg({"Price": "avg"}).collect()[0][0]
cleaned_products_df = cleaned_products_df.withColumn("Price", when(col("Price") <= 0, avg_price).otherwise(col("Price")))
print("completed.")

# Step 9: Export the cleaned data to a CSV file
logger.info("Step 9: Exporting the cleaned data to 'cleaned_products.csv'...")
# save_df_to_csv(cleaned_customers_df, "Cleaned_data/cleaned_products.csv")

# show duplicates
# get_duplicate_data_per_column(cleaned_products_df)

# Display the count of records after phone number processing
record_count_after_cleaning = cleaned_products_df.count()
print(f"Number of records after cleaning: {record_count_after_cleaning}")

logger.info("Data cleaning and export completed successfully.")

2024-09-02 16:49:57,764 - DataCleaningLogger - INFO - Step 1: Counting missing values in each column before filling them...
2024-09-02 16:49:57,865 - DataCleaningLogger - INFO - Step 2: Checking for duplicates in each column before dropping them...


+----------+------------+--------+-----+
|Product_ID|Product_Name|Category|Price|
+----------+------------+--------+-----+
|         0|           0|       3|    0|
+----------+------------+--------+-----+



2024-09-02 16:49:58,630 - DataCleaningLogger - INFO - Step 3: Checking and removing duplicate records based on 'Customer_ID'...
2024-09-02 16:49:58,818 - DataCleaningLogger - INFO - Step 4: Filling missing values in 'Email' and 'Phone' columns...


+------------+---------------+
|      Column|Duplicate_Count|
+------------+---------------+
|  Product_ID|              1|
|Product_Name|              2|
|    Category|              5|
|       Price|              2|
+------------+---------------+

Number of duplicate records before dropping: 1
Number of duplicate records after dropping: 0


2024-09-02 16:49:58,905 - DataCleaningLogger - INFO - Step 5: Capitalizing the first letter of each word in the 'Product_Name' and 'Category' columns...
2024-09-02 16:49:58,913 - DataCleaningLogger - INFO - Step 6: Counting missing values in each column after filling them...
2024-09-02 16:49:59,037 - DataCleaningLogger - INFO - Step 7: Checking for duplicates in each column after dropping them...


+----------+------------+-------------+-----+
|Product_ID|Product_Name|Category     |Price|
+----------+------------+-------------+-----+
|1         |him         |Home         |411  |
|2         |why         |Electronics  |333  |
|3         |strategy    |Home         |645  |
|4         |interview   |Uncategorized|290  |
|5         |though      |Electronics  |1738 |
+----------+------------+-------------+-----+
only showing top 5 rows

Completed.
+----------+------------+--------+-----+
|Product_ID|Product_Name|Category|Price|
+----------+------------+--------+-----+
|         0|           0|       0|    0|
+----------+------------+--------+-----+



2024-09-02 16:49:59,919 - DataCleaningLogger - INFO - Step 8: Replacing negative or zero prices with the average price...
2024-09-02 16:49:59,992 - DataCleaningLogger - INFO - Step 9: Exporting the cleaned data to 'cleaned_products.csv'...
2024-09-02 16:50:00,076 - DataCleaningLogger - INFO - Data cleaning and export completed successfully.


+------------+---------------+
|      Column|Duplicate_Count|
+------------+---------------+
|  Product_ID|              0|
|Product_Name|              1|
|    Category|              5|
|       Price|              1|
+------------+---------------+

completed.
Number of records after cleaning: 50


## 3. Transactions Table

In [217]:
# Step 1: Count missing values in each column before filling them
logger.info("Step 1: Counting missing values in each column before filling them...")
missing_values_before = count_missing_values(transactions_df)
missing_values_before.show()

# Step 2: Calculate the mean value of the 'Amount' column
logger.info("Step 2: Calculating the mean value of the 'Amount' column...")
mean_amount_row = transactions_df.agg(mean("Amount")).collect()[0]
mean_amount = mean_amount_row[0]
print(f"Mean value calculated for 'Amount': {mean_amount:.2f}")

# Step 3: Fill missing values in the 'Amount' column with the rounded mean value
logger.info("Step 3: Filling missing values in the 'Amount' column with the mean value...")
cleaned_transactions_df = fill_missing_values(transactions_df, {"Amount": round(mean_amount, 2)})

# Step 4: Check for duplicate records based on 'Transaction_ID'
logger.info("Step 4: Checking for duplicate records based on 'Transaction_ID'...")
cleaned_transactions_df = drop_duplicates(cleaned_transactions_df, "Transaction_ID")

# Step 5: Ensure 'Date' is in YYYY-MM-DD format
logger.info("Step 5: Ensuring 'Date' column is in YYYY-MM-DD format...")
cleaned_transactions_df = cleaned_transactions_df.withColumn(
    "Date", date_format(col("Date"), "yyyy-MM-dd")
)

# Step 6: date validation 
logger.info("Step 6: Validating dates present in Date column...")
cleaned_transactions_df = date_validation(cleaned_transactions_df,"Date")

# Step 7: Cross-verification after filling missing values in each column
logger.info("Step 7: Counting missing values in each column after filling them...")
missing_values_after = count_missing_values(cleaned_transactions_df)
missing_values_after.show()

# Step 8: Correct inaccurate data (e.g., non-positive 'Amount')
logger.info("Step 8: Correcting non-positive values in the 'Amount' column...")
corrected_transactions_df = cleaned_transactions_df.withColumn(
    "Amount",
    when(col("Amount") <= 0, mean_amount).otherwise(col("Amount"))
)

# Step 9: Display the cleaned DataFrame
logger.info("Step 9: Displaying the cleaned 'transactions_df' DataFrame...")
corrected_transactions_df.show(truncate=False)

# Step 10: Save the cleaned data to a new CSV
logger.info("Step 10: Saving the cleaned data to 'cleaned_transactions.csv'...")
# save_df_to_csv(corrected_transactions_df, "Cleaned_data/cleaned_transactions.csv")

# Display the count of records after phone number processing
record_count_after_cleaning = cleaned_transactions_df.count()
print(f"Number of records after cleaning: {record_count_after_cleaning}")
logger.info("Data cleaning and export completed successfully.")

2024-09-02 16:50:00,090 - DataCleaningLogger - INFO - Step 1: Counting missing values in each column before filling them...
2024-09-02 16:50:00,216 - DataCleaningLogger - INFO - Step 2: Calculating the mean value of the 'Amount' column...
2024-09-02 16:50:00,278 - DataCleaningLogger - INFO - Step 3: Filling missing values in the 'Amount' column with the mean value...
2024-09-02 16:50:00,282 - DataCleaningLogger - INFO - Step 4: Checking for duplicate records based on 'Transaction_ID'...


+--------------+-----------+----------+----+------+------------+
|Transaction_ID|Customer_ID|Product_ID|Date|Amount|Sales_Rep_ID|
+--------------+-----------+----------+----+------+------------+
|             0|          0|         0|   0|    52|           0|
+--------------+-----------+----------+----+------+------------+

Mean value calculated for 'Amount': 282.78
Number of duplicate records before dropping: 30


2024-09-02 16:50:00,533 - DataCleaningLogger - INFO - Step 5: Ensuring 'Date' column is in YYYY-MM-DD format...
2024-09-02 16:50:00,540 - DataCleaningLogger - INFO - Step 6: Validating dates present in Date column...
2024-09-02 16:50:00,663 - DataCleaningLogger - INFO - Step 7: Counting missing values in each column after filling them...


Number of duplicate records after dropping: 0
Step 1: Identifying future dates in 'Interaction_Date'...
No future dates found.


2024-09-02 16:50:00,849 - DataCleaningLogger - INFO - Step 8: Correcting non-positive values in the 'Amount' column...
2024-09-02 16:50:00,859 - DataCleaningLogger - INFO - Step 9: Displaying the cleaned 'transactions_df' DataFrame...
2024-09-02 16:50:00,984 - DataCleaningLogger - INFO - Step 10: Saving the cleaned data to 'cleaned_transactions.csv'...


+--------------+-----------+----------+----+------+------------+
|Transaction_ID|Customer_ID|Product_ID|Date|Amount|Sales_Rep_ID|
+--------------+-----------+----------+----+------+------------+
|             0|          0|         0|   0|     0|           0|
+--------------+-----------+----------+----+------+------------+

+------------------------------------+------------------------------------+----------+----------+------+------------------------------------+
|Transaction_ID                      |Customer_ID                         |Product_ID|Date      |Amount|Sales_Rep_ID                        |
+------------------------------------+------------------------------------+----------+----------+------+------------------------------------+
|00212675-af49-4cbd-9766-4303572b7506|41b14a02-df69-4f06-bba9-a32a32d7c008|30        |2024-05-01|125.0 |61989b73-1807-414a-a75b-f194c0152dba|
|003a04fd-3d61-4af1-bf78-64b4d8bec34b|2fbd8187-90f1-409e-89ff-3ff4cae288e6|14        |2024-01-23|50.0  |51

2024-09-02 16:50:01,081 - DataCleaningLogger - INFO - Data cleaning and export completed successfully.


Number of records after cleaning: 1000


## 4. Interactions Table

In [218]:
# Step 1: Identify missing values in each column before filling them
logger.info("Step 1: Identifying missing values in each column before filling them...")
missing_values_before = count_missing_values(interactions_df)
missing_values_before.show()

# Step 2: Count occurrences of each 'Interaction_Type' and identify the most occurring type
logger.info("Step 2: Counting occurrences of each 'Interaction_Type' to find the most occurring type...")
type_counts = interactions_df.groupBy("Interaction_Type").count() \
    .orderBy(col("count").desc()) \
    .first()

# Extract the most occurring 'Interaction_Type'
most_occurring_type = type_counts["Interaction_Type"] if type_counts else None
print(f"The most occurring 'Interaction_Type' is: {most_occurring_type}")

# Step 3: Replace only null values in 'Interaction_Type' with the most occurring type
logger.info("Step 3: Replacing null values in 'Interaction_Type' with the most occurring type...")
cleaned_interactions_df = interactions_df.withColumn(
    "Interaction_Type",
    when(col("Interaction_Type").isNull(), lit(most_occurring_type))
    .otherwise(col("Interaction_Type"))
)

# Step 4: Check for duplicate records based on 'Interaction_ID'
logger.info("Step 4: Checking for duplicate records based on 'Interaction_ID'...")
cleaned_interactions_df = drop_duplicates(cleaned_interactions_df, "Interaction_ID")

# Step 5: Capitalize the first letter of the values in the 'Issue_Resolved' and 'Interaction_Type' columns
logger.info("Step 5: Capitalizing the first letter of each word in the and 'Interaction_Type' columns...")
cleaned_interactions_df = capitalize_columns(cleaned_interactions_df, [ "Interaction_Type"])
print("Completed.")

# Step 6: date validation 
logger.info("Step 6: Validating dates present in Interaction_Date column...")
cleaned_interactions_df = date_validation(cleaned_interactions_df,"Interaction_Date")

# Step 7: boolean validation  
logger.info("Step 7: Validating booleans present in Issue_Resolved column...")
cleaned_interactions_df = validate_boolean_values(cleaned_interactions_df,"Issue_Resolved")

# Step 8: Cross-verification of missing values in each column after filling them
logger.info("Step 8: Identifying missing values in each column after filling them...")
missing_values_after = count_missing_values(cleaned_interactions_df)
missing_values_after.show()

# Step 9: Display the cleaned 'interactions_df' DataFrame
logger.info("Step 9: Displaying the cleaned 'interactions_df' DataFrame...")
cleaned_interactions_df.show(truncate=False)

# Step 10: Save the cleaned data to a new CSV
logger.info("Step 10: Saving the cleaned data to 'cleaned_interactions.csv'...")
# save_df_to_csv(cleaned_interactions_df, "Cleaned_data/cleaned_interactions.csv")

# Display the count of records after phone number processing
record_count_after_cleaning = cleaned_interactions_df.count()
print(f"Number of records after cleaning: {record_count_after_cleaning}")
logger.info("Data cleaning and export completed successfully.")

2024-09-02 16:50:01,096 - DataCleaningLogger - INFO - Step 1: Identifying missing values in each column before filling them...
2024-09-02 16:50:01,223 - DataCleaningLogger - INFO - Step 2: Counting occurrences of each 'Interaction_Type' to find the most occurring type...


+--------------+-----------+----------------+----------------+--------------+
|Interaction_ID|Customer_ID|Interaction_Date|Interaction_Type|Issue_Resolved|
+--------------+-----------+----------------+----------------+--------------+
|             0|          0|               0|              67|             0|
+--------------+-----------+----------------+----------------+--------------+



2024-09-02 16:50:01,321 - DataCleaningLogger - INFO - Step 3: Replacing null values in 'Interaction_Type' with the most occurring type...
2024-09-02 16:50:01,331 - DataCleaningLogger - INFO - Step 4: Checking for duplicate records based on 'Interaction_ID'...


The most occurring 'Interaction_Type' is: Chat


2024-09-02 16:50:01,516 - DataCleaningLogger - INFO - Step 5: Capitalizing the first letter of each word in the and 'Interaction_Type' columns...
2024-09-02 16:50:01,520 - DataCleaningLogger - INFO - Step 6: Validating dates present in Interaction_Date column...


Number of duplicate records before dropping: 32
Number of duplicate records after dropping: 0
Completed.
Step 1: Identifying future dates in 'Interaction_Date'...


2024-09-02 16:50:01,635 - DataCleaningLogger - INFO - Step 7: Validating booleans present in Issue_Resolved column...
2024-09-02 16:50:01,777 - DataCleaningLogger - INFO - Step 8: Identifying missing values in each column after filling them...


No future dates found.
Step 1: Identifying non-boolean values.. 
All values in 'Issue_Resolved' are valid booleans.


2024-09-02 16:50:01,970 - DataCleaningLogger - INFO - Step 9: Displaying the cleaned 'interactions_df' DataFrame...
2024-09-02 16:50:02,049 - DataCleaningLogger - INFO - Step 10: Saving the cleaned data to 'cleaned_interactions.csv'...
2024-09-02 16:50:02,148 - DataCleaningLogger - INFO - Data cleaning and export completed successfully.


+--------------+-----------+----------------+----------------+--------------+
|Interaction_ID|Customer_ID|Interaction_Date|Interaction_Type|Issue_Resolved|
+--------------+-----------+----------------+----------------+--------------+
|             0|          0|               0|               0|             0|
+--------------+-----------+----------------+----------------+--------------+

+------------------------------------+------------------------------------+----------------+----------------+--------------+
|Interaction_ID                      |Customer_ID                         |Interaction_Date|Interaction_Type|Issue_Resolved|
+------------------------------------+------------------------------------+----------------+----------------+--------------+
|002debb8-7e9e-476c-85bc-dd3e8b7210ce|dfce9c24-71c4-4cd8-9b59-f34e48842979|2024-01-30      |Chat            |true          |
|00571a5f-03ff-4615-85cf-cd540f2ccdad|bd56525d-e899-4312-9763-e79d3f6fa54c|2024-02-08      |Chat            |

## 5. Sales Team Table 

In [219]:
# Step 1: Identify Missing Values
logger.info("Step 1: Identifying missing values in each column before filling them...")
missing_values_before = count_missing_values(sales_team_df)
missing_values_before.show()

# Step 2: Handle Missing Values
logger.info("Step 2: Handling missing values by filling with averages...")
# Calculate average Sales_Achieved
avg_sales_achieved = sales_team_df.select(avg(col("Sales_Achieved"))).first()[0]
print(f"Average value of sales achieved column is {avg_sales_achieved}")

# Fill missing Sales_Achieved with the average
sales_team_cleaned_df = fill_missing_values(sales_team_df,{"Sales_Achieved": round(avg_sales_achieved,2)})

# Step 3: Check for Duplicate Values
logger.info("Step 3: Checking for duplicate records based on 'Sales_Rep_ID'...")
duplicate_count_before = count_duplicates_per_column(sales_team_cleaned_df)

# Step 4: Drop Duplicates if they exist
sales_team_cleaned_df = drop_duplicates(sales_team_cleaned_df, "Sales_Rep_ID")

# Step 6: Standardize Formats
logger.info("Step 6: Standardizing the format of 'Name' and 'Region' columns...")
sales_team_cleaned_df = capitalize_columns(sales_team_cleaned_df, [ "Name", "Region"])
print("Completed.")

# Step 7: Cross-verification of missing values in each column after filling them
logger.info("Step 7: Identifying missing values in each column after filling them...")
missing_values_after = count_missing_values(sales_team_cleaned_df)
missing_values_after.show()

# Step 8: Display the cleaned 'sales_team_df' DataFrame
logger.info("Step 8: Displaying the cleaned 'sales_team_df' DataFrame...")
sales_team_cleaned_df.show(truncate=False)

# Step 9: Save the cleaned data to a new CSV
logger.info("Step 9: Saving the cleaned data to 'cleaned_sales_team.csv'...")
# save_df_to_csv(sales_team_cleaned_df, "Cleaned_data/cleaned_sales_team.csv")

# Display the count of records after phone number processing
record_count_after_cleaning = sales_team_cleaned_df.count()
print(f"Number of records after cleaning: {record_count_after_cleaning}")
logger.info("Data cleaning and export completed successfully.")

2024-09-02 16:50:02,164 - DataCleaningLogger - INFO - Step 1: Identifying missing values in each column before filling them...
2024-09-02 16:50:02,267 - DataCleaningLogger - INFO - Step 2: Handling missing values by filling with averages...
2024-09-02 16:50:02,327 - DataCleaningLogger - INFO - Step 3: Checking for duplicate records based on 'Sales_Rep_ID'...


+------------+----+------+------------+--------------+
|Sales_Rep_ID|Name|Region|Sales_Target|Sales_Achieved|
+------------+----+------+------------+--------------+
|           0|   0|     0|           0|             5|
+------------+----+------+------------+--------------+

Average value of sales achieved column is 21912.425531914894


2024-09-02 16:50:02,938 - DataCleaningLogger - INFO - Step 6: Standardizing the format of 'Name' and 'Region' columns...
2024-09-02 16:50:02,949 - DataCleaningLogger - INFO - Step 7: Identifying missing values in each column after filling them...


Number of duplicate records before dropping: 2
Number of duplicate records after dropping: 0
Completed.


2024-09-02 16:50:03,088 - DataCleaningLogger - INFO - Step 8: Displaying the cleaned 'sales_team_df' DataFrame...
2024-09-02 16:50:03,226 - DataCleaningLogger - INFO - Step 9: Saving the cleaned data to 'cleaned_sales_team.csv'...


+------------+----+------+------------+--------------+
|Sales_Rep_ID|Name|Region|Sales_Target|Sales_Achieved|
+------------+----+------+------------+--------------+
|           0|   0|     0|           0|             0|
+------------+----+------+------------+--------------+

+------------------------------------+---------------------+--------------+------------+--------------+
|Sales_Rep_ID                        |Name                 |Region        |Sales_Target|Sales_Achieved|
+------------------------------------+---------------------+--------------+------------+--------------+
|02acdbab-d149-4053-8c7a-871ba76f003e|Daniel Barber        |Arizona       |13651       |7296.0        |
|0437b05a-9628-43f9-ac07-0b9a0dc96dcd|Brittany Taylor      |California    |41135       |14037.0       |
|0468e32e-644b-491f-a3e1-741306f2c3f2|Dr. Wayne Spencer Dvm|Pennsylvania  |15593       |37653.0       |
|05cc8513-8a8a-4598-862a-218d1093ae26|Pamela Pennington    |Idaho         |38399       |11569.0     

2024-09-02 16:50:03,295 - DataCleaningLogger - INFO - Data cleaning and export completed successfully.


Number of records after cleaning: 50
